# CPS Report

### Preliminary installations with ROS Noetic/Ubuntu 20.04 

https://docs.px4.io/main/en/dev_setup/dev_env_linux_ubuntu.html

In [ ]:
# download PX4
!git clone https://github.com/PX4/PX4-Autopilot.git --recursive

# install
!bash ./PX4-Autopilot/Tools/setup/ubuntu.sh -no-sim-tools --no-nuttx

# then restart computer

# additional dependencies
!sudo apt-get install protobuf-compiler libeigen3-dev libopencv-dev -y

http://wiki.ros.org/noetic/Installation/Ubuntu#Installation

In [ ]:
# ROS Noetic installation
!sudo sh -c 'echo "deb http://packages.ros.org/ros/ubuntu $(lsb_release -sc) main" > /etc/apt/sources.list.d/ros-latest.list'
!sudo apt install curl # if you haven't already installed curl
!curl -s https://raw.githubusercontent.com/ros/rosdistro/master/ros.asc | sudo apt-key add -

# update packages
!sudo apt update

# ROS desktop full install
!sudo apt install ros-noetic-desktop-full

# to install specific packages:
!sudo apt install ros-noetic-PACKAGE # ex: sudo apt install ros-noetic-slam-gmapping

In [ ]:
# you must source this script in every bash terminal you use ROS in
!source /opt/ros/noetic/setup.bash

# or you can automatically source it every time a terminal is launched (do it just once):
!echo "source /opt/ros/noetic/setup.bash" >> ~/.bashrc
!source ~/.bashrc

In [ ]:
# dependencies for building packages:
!sudo apt install python3-rosdep python3-rosinstall python3-rosinstall-generator python3-wstool build-essential

# initialize rosdep
!sudo rosdep init
!rosdep update

https://docs.px4.io/main/en/ros/mavros_installation.html

In [ ]:
# Mavros installation
!